# 인프라 데이터 통합 및 PNU 생성

다음 셀은 워크스페이스의 `공원녹지`, `교육인프라`, `유통시설` 폴더에 있는 시기별 CSV 파일들을 읽어 통합하고,
`HONO_NM`(번지)에서 `본번`/`부번`을 파싱해 `pnu` 및 19자리 `PNU` 컬럼을 생성합니다.
PNU 구성: `11500103001`(공통) + 본번(4자리, 왼쪽0채움) + 부번(4자리, 왼쪽0채움)

**작업 요약**

- **목표:** 매매·전월세·표제부 + 인프라(공원녹지, 교육인프라, 유통시설) 데이터를 통합하여 마스터 테이블 생성.  
- **노트북:** `공원녹지 통합.ipynb`에 코드 및 설명 추가(인프라 통합, PNU 생성, 필터링, 마스터 병합 스크립트 포함).

**세부 단계(실행된 작업)**

- **1. 워크스페이스 점검:**  
  - 인프라 폴더: 공원녹지, 교육인프라, 유통시설 내부 CSV 확인.

- **2. 인프라 통합 및 PNU 생성:**  
  - 각 폴더의 시기별 CSV들을 합쳐 `*_combined.csv` 생성.  
  - 번지(`HONO_NM`) 파싱: `bonbun`, `bubun` 추출 → `bonbun4`/`bubun4`로 4자리 0채움 → `PNU` = `11500103001` + `bonbun4` + `bubun4` (19자리).  
  - 중복 소문자 `pnu` 컬럼이 있으면 삭제하고 `PNU`만 유지.

- **3. 인프라별 결과(파일 및 요약):**  
  - 공원녹지_combined.csv — rows=1,199,502, cols=29, unique PNU=30,036  
  - 교육인프라_combined.csv — rows=599,751, cols=29, unique PNU=30,036  
  - 유통시설_combined.csv — rows=599,751, cols=29, unique PNU=30,036

- **4. 특정 동 필터링(요청)**  
  - 각 인프라 결합 파일에서 `EMD_NM == '화곡동'` 필터 후 저장:  
    - 공원녹지_화곡동.csv — rows=27,562  
    - 교육인프라_화곡동.csv — rows=13,782  
    - 유통시설_화곡동.csv — rows=13,782

- **5. 마스터 결합(매매·전월세·표제부 + 인프라):**  
  - 불러온 파일: `최종) 매매_전처리.csv`, `최종) 전월세_전처리.csv`, `03. 표제부_20260122144509.csv`  
  - 전월세: `PNU`별 집계(갯수, 평균 보증금/월세, 최근 계약일) 생성(컬럼명 접두어로 충돌 회피).  
  - 표제부: `새주소본번`/`새주소부번`(또는 `번`/`지`)로 `PNU` 생성 후 `PNU` 기준으로 주요 표제부 컬럼만 선택(중복 제거).  
  - 인프라: 각 `*_combined.csv`에서 `PNU`별로 최소거리(`*_min_dist`)와 카운트(`*_count`) 집계.  
  - 병합 순서: 매매(기준, left) ← 전월세요약 ← 표제부요약 ← 인프라집계 (left-join).  
  - 결과 파일: master_combined.csv — rows=4,748, cols=41

**컬럼 처리 및 중복 규칙 요약**

- **PNU 처리:** 인프라에서 생성한 `PNU`를 기준으로 통합. 중복 `pnu` 컬럼이 있으면 삭제하고 `PNU`만 유지.  
- **집계 컬럼 네이밍:** 전월세·인프라 집계는 접두사(`rent_`, `park_`, `edu_`, `dist_`)를 붙여 병합 시 이름 충돌을 피함.  
- **표제부:** `PNU` 단위로 중복 제거(`drop_duplicates(subset=['PNU'])`) 후 필요한 열만 선택해 병합.  
- **데이터 타입 정리:** 숫자형(예: `거래금액(만원)`, `보증금`)은 `,` 제거 후 `to_numeric`, 날짜는 `to_datetime(..., errors='coerce')` 적용.  
- **남은 충돌:** 병합으로 `_x`/`_y` 접미사가 생기면 필요시 정리할 수 있음(현재는 의도적으로 최소화).

**생성된 주요 파일(위치)**

- 공원녹지_combined.csv  
- 교육인프라_combined.csv  
- 유통시설_combined.csv  
- 공원녹지_화곡동.csv  
- 교육인프라_화곡동.csv  
- 유통시설_화곡동.csv  
- master_combined.csv

**권장되는 다음 단계 (선택)**

- `_x/_y` 접미사 컬럼 정리 및 불필요 원본 컬럼 제거.  
- master_combined.csv에 대한 변수 분포/결측 요약(예: `거래금액(만원)`, `보증금(만원)`, `건축연령` 등).  
- VIF 제거 및 Stepwise(AIC/BIC/p-value) 기반 변수 선택 준비(모델용 전처리 스크립트 추가).  
- 헤도닉 모델(적정가·잔차·매매-전세 시차) 구현 시작.

원하시면 바로 (1) `_x/_y` 정리 및 마스터 컬럼 정리, (2) master_combined.csv의 변수별 요약(기술통계 + 히스토그램 샘플), 또는 (3) VIF 계산용 전처리(결측·스케일링) 중 어느 것을 먼저 할지 알려주세요.

In [ ]:
import pandas as pd
import glob, os, re

BASE = '/Users/lopi/Downloads/pj'
INFRA_FOLDERS = {
    '공원녹지': '공원녹지',
    '교육인프라': '교육인프라',
    '유통시설': '유통시설',
}
COMMON_PNU_PREFIX = '11500103001'

def load_and_concat(folder_name):
    pattern = os.path.join(BASE, folder_name, '*.csv')
    files = sorted(glob.glob(pattern))
    dfs = []
    for f in files:
        # CSVs use '|' as separator in this project
        df = pd.read_csv(f, sep='|', dtype=str, encoding='utf-8', low_memory=False)
        df['__source_file'] = os.path.basename(f)
        dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()

def parse_hono(h):
    if pd.isna(h):
        return (None, None)
    s = str(h).strip()
    # match patterns like '1163-4' or '1164'
    m = re.match(r'^(\d+)(?:-(\d+))?$', s)
    if m:
        bon = m.group(1)
        bub = m.group(2) or '0'
        return (bon, bub)
    # fallback: extract first two numeric groups if present
    parts = re.findall(r'\d+', s)
    if len(parts) == 0:
        return (None, None)
    if len(parts) == 1:
        return (parts[0], '0')
    return (parts[0], parts[1])

# 로드 및 파싱 실행 (메모리 상황에 유의)
infra_dfs = {}
for label, folder in INFRA_FOLDERS.items():
    print(f'Loading {label} from {folder}...')
    df = load_and_concat(folder)
    if df.empty:
        print(f'  No files found for {label}')
        infra_dfs[label] = df
        continue
    # ensure HONO_NM exists and parse it
    if 'HONO_NM' not in df.columns:
        print(f"  Warning: 'HONO_NM' not in columns for {label}")
    df['HONO_NM'] = df.get('HONO_NM', pd.NA)
    parsed = df['HONO_NM'].apply(parse_hono)
    df['bonbun'] = parsed.apply(lambda x: x[0] if x else None)
    df['bubun'] = parsed.apply(lambda x: x[1] if x else None)
    df['bonbun4'] = df['bonbun'].fillna('0').astype(str).str.zfill(4)
    df['bubun4'] = df['bubun'].fillna('0').astype(str).str.zfill(4)
    df['pnu'] = COMMON_PNU_PREFIX + df['bonbun4'] + df['bubun4']
    df['PNU'] = df['pnu']
    infra_dfs[label] = df
    out_path = os.path.join(BASE, f'{label}_combined.csv')
    print(f'  saving combined -> {out_path} (rows={len(df)})')
    df.to_csv(out_path, index=False, sep='|')

# 노트북에서 이어서 확인할 수 있도록 infra_dfs 변수 유지
infra_dfs.keys()

In [ ]:
# 각 통합 데이터의 상위 몇 줄을 확인
for k, df in infra_dfs.items():
    print('---', k, '---')
    display(df.head(3))